In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from seaborn import heatmap
import matplotlib.pyplot as plt
from datetime import time, datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

In [ ]:
df['Dates'] = pd.to_datetime(df['timestamp']).dt.date
df['Time'] = pd.to_datetime(df['timestamp']).dt.time


In [ ]:
_df= df.drop(columns=['ID', 'Bump', 'Roundabout', 'timestamp'])
_df

In [ ]:
_df['Crossing'] = _df['Crossing'].astype('int')
_df['Give_Way'] = _df['Give_Way'].astype('int')
_df['Junction'] = _df['Junction'].astype('int')
_df['No_Exit'] = _df['No_Exit'].astype('int')
_df['Railway'] = _df['Railway'].astype('int')
_df['Stop'] = _df['Stop'].astype('int')
_df['Amenity'] = _df['Amenity'].astype('int')
enc = LabelEncoder()
enc.fit(_df['Side'])
_df['Side'] = enc.transform(_df['Side'])
_df

In [ ]:
df_corr = _df.corr()
df_corr

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))
heatmap(df_corr, cmap="gray")
plt.show()

In [ ]:
a= time(hour = 7, minute = 0, second = 0)
b= time(hour = 14, minute = 0, second = 0)
c= time(hour = 0, minute = 0, second = 0)

_df['Time_Interval'] = _df['Time'].apply(lambda x: 'Dawn' if (x <= a and x > c) else ('Day' if (x <= b and x > a) else 'Night'))

In [ ]:
_df.groupby(['Time_Interval']).count()

In [ ]:
_df['Time']

In [ ]:
df2 = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

In [ ]:
df2['newDate'] = df2.apply(lambda row: datetime.strptime(f"{int(row.Year)}-{int(row.Month)}-{int(row.Day)}", '%Y-%m-%d'), axis=1)

In [ ]:
_df2= df2.drop(columns=['Year', 'Day', 'Selected'])
_df2

In [ ]:
_df2['newHour'] = _df2.apply(lambda row: datetime.strptime(f"{int(row.Hour)}:{'00'}:{'00'}", '%H:%M:%S'), axis=1)
_df2

In [ ]:
_df2['Time'] = pd.to_datetime(_df2['newHour']).dt.time
_df2['Time']

In [ ]:
_df2= _df2.drop(columns=['Hour', 'newHour'])

In [ ]:
a= time(hour = 7, minute = 0, second = 0)
b= time(hour = 14, minute = 0, second = 0)
c= time(hour = 0, minute = 0, second = 0)

_df2['Time_Interval'] = _df2['Time'].apply(lambda x: 'Dawn' if (x <= a and x > c) else ('Day' if (x <= b and x > a) else 'Night'))

In [ ]:
_df2['Temperature(F)'].isnull().sum(axis = 0)  #Drop 1775, 6539

In [ ]:
_df2['Humidity(%)'].isnull().sum(axis = 0)   #Drop 1775, 6539

In [ ]:
_df2['Precipitation(in)'].isnull().sum(axis = 0)

In [ ]:
_df2['Weather_Condition'].isnull().sum(axis = 0)   #Drop 1775

In [ ]:
_df2['Wind_Chill(F)'].isnull().sum(axis = 0)

In [ ]:
_df2['Wind_Speed(mph)'].isnull().sum(axis = 0)

In [ ]:
_df2['Visibility(mi)'].isnull().sum(axis = 0)   #Drop 1775

In [ ]:
df2_corr = _df2.corr()
df2_corr

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))
heatmap(df2_corr, cmap="gray")
plt.show()

In [ ]:
def imputation_Vals(X, imputed_Col_name):
    """This function returns the Mean, Median, and Mode"""
    X_sort= X.sort_values('Month')
    _X_sort= X_sort[X_sort[f"{imputed_Col_name}"].notna()]
    mean_arr= []
    median_arr= []
    mode_arr= []
    for i in range(12):
        temp= _X_sort.loc[_X_sort['Month'] == (i+1)]
        mean_ele= temp[f"{imputed_Col_name}"].mean()
        mean_arr.append(mean_ele)
        median_ele= temp[f"{imputed_Col_name}"].median()
        median_arr.append(median_ele)
        mode_ele= temp[f"{imputed_Col_name}"].mode()
        mode_arr.append(mode_ele)
    
    return mean_arr, median_arr, mode_arr
        

In [ ]:
def imputation_Imp(A, DF, imputed_Col_name):
    """This function applies Mean or Median or Mode to NaN values"""
    X_sort= DF.sort_values('Month')
    _X_sort= X_sort[X_sort[f"{imputed_Col_name}"].isnull()]
    index= [1,2,3,4,5,6,7,8,9,10,11,12]
    dict_of_arr= dict(zip(index, A))
    _X_sort_NEW= _X_sort['Month'].map(dict_of_arr)
    
    return _X_sort_NEW


In [ ]:
_df2_Precipitation= _df2[['Month', 'Precipitation(in)']]

In [ ]:
_df2_Wind_Speed= _df2[['Month', 'Wind_Speed(mph)']]
_df2_Wind_Speed

In [ ]:
_df2_Chill= _df2[['Month', 'Wind_Chill(F)']]
_df2_Chill

In [ ]:
MEAN_pre, MEDIAN_pre, MODE_pre= imputation_Vals(_df2_Precipitation, 'Precipitation(in)')

In [ ]:
NEW_VALUE_pre= imputation_Imp(MEAN_pre, _df2_Precipitation, 'Precipitation(in)')

In [ ]:
_df2['Temp_col']= NEW_VALUE_pre

In [ ]:
_df2["new_Precipitation"] = _df2[["Precipitation(in)", "Temp_col"]].sum(axis=1)

In [ ]:
_df2.set_index('newDate')['Precipitation(in)'].plot(figsize = (16,6))

In [ ]:
_df2.set_index('newDate')['new_Precipitation'].plot(figsize = (16,6))

In [ ]:
MEAN_WS, MEDIAN_WS, MODE_WS= imputation_Vals(_df2_Wind_Speed, 'Wind_Speed(mph)')

In [ ]:
NEW_VALUE_WS= imputation_Imp(MEDIAN_WS, _df2_Wind_Speed, 'Wind_Speed(mph)')

In [ ]:
_df2['Temp_col']= NEW_VALUE_WS

In [ ]:
_df2["new_Wind_Speed"] = _df2[["Wind_Speed(mph)", "Temp_col"]].sum(axis=1)

In [ ]:
_df2.set_index('newDate')['Wind_Speed(mph)'].plot(figsize = (16,6))

In [ ]:
_df2.set_index('newDate')['new_Wind_Speed'].plot(figsize = (16,6))

In [ ]:
MEAN_WC, MEDIAN_WC, MODE_WC= imputation_Vals(_df2_Chill, 'Wind_Chill(F)')

In [ ]:
NEW_VALUE_WC= imputation_Imp(MEDIAN_WC, _df2_Chill, 'Wind_Chill(F)')

In [ ]:
_df2['Temp_col']= NEW_VALUE_WC

In [ ]:
_df2["new_Wind_Chill"] = _df2[["Wind_Chill(F)", "Temp_col"]].sum(axis=1)

In [ ]:
_df2.set_index('newDate')['Wind_Chill(F)'].plot(figsize = (16,6))
#Odd distribution

In [ ]:
_df2.set_index('newDate')['new_Wind_Chill'].plot(figsize = (16,6))

In [ ]:
_df2= _df2.drop(columns=['Temp_col'])

In [ ]:
_df['Dates']= pd.to_datetime(_df['Dates'])

In [ ]:
_df.dtypes

In [ ]:
_df2.dtypes

In [ ]:
_DF= _df.merge(_df2, left_on= ['Dates', 'Time_Interval'], right_on= ['newDate', 'Time_Interval'], how= 'left')

In [ ]:
Holiday = pd.read_csv(os.path.join('/kaggle/input/holidayincsv/holidays.csv'))

In [ ]:
df_Holiday= pd.DataFrame(Holiday)
df_Holiday

In [ ]:
df_Holiday['date']=pd.to_datetime(df_Holiday['date']).dt.date

In [ ]:
df_Holiday['date']= pd.to_datetime(df_Holiday['date'])
df_Holiday.dtypes

In [ ]:
Final_DF= _DF.merge(df_Holiday, left_on= ['Dates'], right_on= ['date'], how= 'left')

In [ ]:
train_df, val_df = train_test_split(Final_DF, test_size=0.2, random_state=42)

In [ ]:
X_train = train_df.drop(columns=['Severity'])

In [ ]:
y_train = train_df['Severity']

In [ ]:
X_val = val_df.drop(columns=['Severity'])

In [ ]:
y_val = val_df['Severity']

In [ ]:
X_train = X_train
X_val = X_val

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
classifier = classifier.fit(X_train, y_train)

In [ ]:
classifier.score(X_val, y_val)

In [ ]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))

In [ ]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

In [ ]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)